In [6]:
import pandas as pd
import re

In [7]:
df = pd.read_csv('/spam.csv', encoding='latin1',  index_col=False)

In [ ]:
for index, row in df.iterrows():
  if row['v1'] == 'ham':
    with open('/ham.txt', 'a') as file:
      file.write(row['v2'] + ' ')
  else:
    with open('/spam.txt', 'a') as file:
      file.write(row['v2'] + ' ')

In [9]:
class unique_words_lower_case():

  def __init__(self, ham_text_file, spam_text_file):
    self.ham_text_file = ham_text_file
    self.spam_text_file = spam_text_file

    self.ham_text, self.ham_word_count, self.ham_sentence_count = self.extract_text(self.ham_text_file)
    self.spam_text, self.spam_word_count, self.spam_sentence_count = self.extract_text(self.spam_text_file)

    self.join_text = self.ham_text + ' ' + self.spam_text
    self.ham_unique_words = self.unique_words(self.ham_text)

    self.unique_words = self.unique_words(self.join_text)

    self.ham_word_percentage = self.word_percentage(self.ham_text, self.ham_word_count)
    self.spam_word_percentage = self.word_percentage(self.spam_text, self.spam_word_count)

  def extract_text(self, text_file):
    with open(text_file, 'r') as file:
      text = file.read()
      text = text.lower()
    sentence = len(text.lower().split('. '))
    word_count = len(text.lower().split())
    return text, word_count, sentence

  def unique_words(self, text):
    words = text.lower().split()
    unique_words = []

    for word in words:
      if re.match("\W([A-Za-z]*)", word):
        word = re.match("\W([A-Za-z]*)", word).group(1)

      elif re.match("([A-Za-z]*)\W", word):
        word = re.match("([A-Za-z]*)\W", word).group(1)

      word = word + ' '
      unique_words.append(word)

    unique_words = set(unique_words)
    return unique_words

  def word_percentage(self, text, word_count):
    percent_dict = {}
    for word in self.unique_words:
      key = word.rstrip()
      percent_dict[key] = (text.replace('.', ' ').lower().count(word) + 1) / word_count
    return percent_dict

  def predict_lebel(self, text):
    ham_probability = self.ham_sentence_count / (self.ham_sentence_count + self.spam_sentence_count)
    spam_probability = self.spam_sentence_count / (self.ham_sentence_count + self.spam_sentence_count)
    for word in text.lower().split():
      ham_probability *= self.ham_word_percentage[word]
      spam_probability *= self.spam_word_percentage[word]

    if ham_probability > spam_probability:
      return 'Ham'
    else:
      return 'Spam'

In [10]:
string_to_unique_words = unique_words_lower_case('/ham.txt', '/spam.txt')

In [11]:
print(string_to_unique_words.predict_lebel('''Free entry in 2 a wkly comp to win FA Cup final tkts'''))
print(string_to_unique_words.predict_lebel('''Available only in bugis n great world la e buffet'''))

Spam
Ham
